In [3]:
import tablib

In [4]:
# 创建数据集
data = tablib.Dataset()
# 添加行
names = ['Kenneth Reitz', 'Bessie Monke']

for name in names:
    # split name appropriately
    fname, lname = name.split()
    # add names to Dataset
    data.append([fname, lname])
print(data.dict)

[['Kenneth', 'Reitz'], ['Bessie', 'Monke']]


In [5]:
# 添加标题
data.headers = ['First Name', 'Last Name']
print(data.dict)

[OrderedDict([('First Name', 'Kenneth'), ('Last Name', 'Reitz')]), OrderedDict([('First Name', 'Bessie'), ('Last Name', 'Monke')])]


In [6]:
# 添加列
## 现在我们有了一个基本的 Dataset , 对它添加一列 ages ：
data.append_col([22, 20], header='Age')
print(data.dict)

[OrderedDict([('First Name', 'Kenneth'), ('Last Name', 'Reitz'), ('Age', 22)]), OrderedDict([('First Name', 'Bessie'), ('Last Name', 'Monke'), ('Age', 20)])]


In [7]:
# 导入数据
## 通过导入预先存在的文件创建一个 tablib.Dataset 对象：
with open('data.csv', 'r') as fh:
    imported_data = tablib.Dataset().load(fh)

## 当格式为 csv ， tsv ， dbf ， xls 或 xlsx ，并且数据源没有表头，则应按如下方式导入：
with open('data.csv', 'r') as fh:
    imported_data = tablib.Dataset().load(fh, headers=False)
print(imported_data.dict)

[['123', '1231241', '123123']]


In [8]:
# 导出数据 --功能是将 Dataset 对象转换为多种格式。

## 转 csv 格式：
data.export('csv')

'First Name,Last Name,Age\r\nKenneth,Reitz,22\r\nBessie,Monke,20\r\n'

In [9]:
## 转 json 格式：
data.export('json')

'[{"First Name": "Kenneth", "Last Name": "Reitz", "Age": 22}, {"First Name": "Bessie", "Last Name": "Monke", "Age": 20}]'

In [10]:
## yaml 格式：
data.export('yaml')

'- {Age: 22, First Name: Kenneth, Last Name: Reitz}\n- {Age: 20, First Name: Bessie, Last Name: Monke}\n'

In [11]:
## 转 Microsoft Excel 格式：
data.export('xls')

b'\xd0\xcf\x11\xe0\xa1\xb1\x1a\xe1\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00>\x00\x03\x00\xfe\xff\t\x00\x06\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\t\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\xfe\xff\xff\xff\x00\x00\x00\x00\xfe\xff\xff\xff\x00\x00\x00\x00\x08\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\

In [12]:
## 转 DataFrame 格式：
data.export('df')

,First Name,Last Name,Age
0,Kenneth,Reitz,22
1,Bessie,Monke,20


In [13]:
# 选择行和列
## 可以像标准的Python列表一样切片和访问行。
print(data[0])

## 通过列名访问列
print(data['First Name'])

## 查看列名，通过索引访问列
print(data.headers)
print(data.get_col(1))

# 列的结果以列表形式给出，可进一步对其处理，如计算平均年龄。
ages = data['Age']
print(float(sum(ages)) / len(ages))

('Kenneth', 'Reitz', 22)
['Kenneth', 'Bessie']
['First Name', 'Last Name', 'Age']
['Reitz', 'Monke']
21.0


In [14]:
# 动态列 --动态列是单个可调用对象（ e.g. 函数）。
## 在 Dataset 对象中插入动态列的方法如下，其中函数 random_grade 生成一个随机的分数。
import random
def random_grade(row):
    return random.randint(60,100)
data.append_col(random_grade, header='Grade')
print(data.export('df'))

## 添加动态列时，传递给给定可调用项的第一个参数是当前数据行。可以使用此方法对数据行执行计算。
## 例如，我们可以使用行中可用的数据来猜测学生的性别。
def guess_gender(row):
    """Calculates gender of given student data row."""
    m_names = ('Kenneth', 'Mike', 'Yuri')
    f_names = ('Bessie', 'Samantha', 'Heather')
    name = row[0]
    if name in m_names:
        return 'Male'
    elif name in f_names:
        return 'Female'
    else:
        return 'Unknown'
## 将此函数作为动态列添加到数据集中：
data.append_col(guess_gender, header="Gender")
print(data.export('df'))


  First Name Last Name  Age  Grade
0    Kenneth     Reitz   22    100
1     Bessie     Monke   20     88
  First Name Last Name  Age  Grade  Gender
0    Kenneth     Reitz   22    100    Male
1     Bessie     Monke   20     88  Female


In [17]:
# 使用标记筛选数据集
## 在构造 Dataset 对象时，可以通过指定 tags 参数对 Dataset 进行筛选。这对于基于任意标准分离数据行是有用的。 e.g. 构造以下 Dataset.

students = tablib.Dataset()
students.headers = ['first', 'last']
students.rpush(['Kenneth', 'Reitz'], tags=['male', 'technical'])
students.rpush(['Daniel', 'Dupont'], tags=['male', 'creative' ])
students.rpush(['Bessie', 'Monke'], tags=['female', 'creative'])
print("[1]:\n",students.df)

## 现在在行上有额外的元数据，可以轻松地对数据 Dataset 过滤. 例如查看男学生:
print("[2]:\n",students.filter(['male']).df)

## 默认情况下，当传递标记列表时，将按照逻辑”或“获得过滤数据集。
print("[3]:\n",students.filter(['famale','creative']).df)

## 按如下方式可以按照逻辑”与“获得过滤数据集。
print("[4]:\n",students.filter('male').filter('creative').json)

## 以上操作中，原始的 Dataset 没有被更改。
print("[5]:\n",students.df)

[1]:
      first    last
0  Kenneth   Reitz
1   Daniel  Dupont
2   Bessie   Monke
[2]:
      first    last
0  Kenneth   Reitz
1   Daniel  Dupont
[3]:
     first    last
0  Daniel  Dupont
1  Bessie   Monke
[4]:
 [{"first": "Daniel", "last": "Dupont"}]
[5]:
      first    last
0  Kenneth   Reitz
1   Daniel  Dupont
2   Bessie   Monke
